In [55]:
import numpy as np
import pandas as pan
from sklearn import linear_model
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
import math


# Load the diabetes dataset
head_bias = ['dist', 'amp','1', '2', '3', '4', '5', '6', '7', '8', '9', 'id']
head_bias1 = ['dist', 'amp','1', '2', '3', '4', '5', '6', '7', '8', '9', 'id']
res = pan.read_csv("/Users/pietrodimarco/IdeaProjects/fittpy/results.csv",names=head_bias)
res2 = pan.read_csv("/Users/pietrodimarco/IdeaProjects/fittpy/results2.csv",names=head_bias1)
result = pan.merge(res, res2, on=['dist', 'amp'])
result.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 22 columns):
dist    6 non-null float64
amp     6 non-null float64
1_x     6 non-null int64
2_x     6 non-null int64
3_x     6 non-null int64
4_x     6 non-null int64
5_x     6 non-null int64
6_x     6 non-null int64
7_x     6 non-null int64
8_x     6 non-null int64
9_x     6 non-null int64
id_x    6 non-null int64
1_y     6 non-null int64
2_y     6 non-null int64
3_y     6 non-null int64
4_y     6 non-null int64
5_y     6 non-null int64
6_y     6 non-null int64
7_y     6 non-null int64
8_y     6 non-null int64
9_y     6 non-null int64
id_y    6 non-null int64
dtypes: float64(2), int64(20)
memory usage: 1.1 KB


In [56]:

result=result.drop(['id_x','id_y'],axis=1)
result=result.assign(id = lambda x: np.log(x.dist/x.amp+1))

In [57]:
from collections import defaultdict
trainX=list()
trainY=list()
avgThr=list()
for row in result.itertuples():
    sum=0
    for i in range(2, len(row)-2):
        sum+=row[len(row)-1]/(row[i]/1000)
        trainY.append(row[i]/1000)
        trainX.append(row[len(row)-1])
    avgThr.append(sum/18)
            
trainY = np.asarray(trainY)
trainX = np.asarray(trainX)

print(trainX)
print(trainY)


[ 3.71357207  3.71357207  3.71357207  3.71357207  3.71357207  3.71357207
  3.71357207  3.71357207  3.71357207  3.71357207  3.71357207  3.71357207
  3.71357207  3.71357207  3.71357207  3.71357207  3.71357207  3.71357207
  3.04452244  3.04452244  3.04452244  3.04452244  3.04452244  3.04452244
  3.04452244  3.04452244  3.04452244  3.04452244  3.04452244  3.04452244
  3.04452244  3.04452244  3.04452244  3.04452244  3.04452244  3.04452244
  2.83321334  2.83321334  2.83321334  2.83321334  2.83321334  2.83321334
  2.83321334  2.83321334  2.83321334  2.83321334  2.83321334  2.83321334
  2.83321334  2.83321334  2.83321334  2.83321334  2.83321334  2.83321334
  2.19722458  2.19722458  2.19722458  2.19722458  2.19722458  2.19722458
  2.19722458  2.19722458  2.19722458  2.19722458  2.19722458  2.19722458
  2.19722458  2.19722458  2.19722458  2.19722458  2.19722458  2.19722458
  3.93182563  3.93182563  3.93182563  3.93182563  3.93182563  3.93182563
  3.93182563  3.93182563  3.93182563  3.93182563  3

In [58]:
regr = linear_model.LinearRegression()
regr.fit(trainX.reshape(108,1),trainY.reshape(108,1))
mean=pan.DataFrame(columns=['id','avgThr'])
for first,second in zip(avgThr,result['id']):
    mean.loc[len(mean)]=[second,first]
mean

,id,avgThr
0,3.713572,24.163553
1,3.044522,9.890516
2,2.833213,9.995480
3,2.197225,7.984676
4,3.931826,25.196225
5,3.044522,20.444748


In [59]:

mean_dic = defaultdict(list)
for row in mean.itertuples():
    mean_dic[row[1]].append(row[2])

In [60]:
pred = regr.predict(trainX.reshape(108,1))
print('Coefficient: ',regr.coef_)
print('intercept: ',regr.intercept_)

Coefficient:  [[ 0.23910331]]
intercept:  [ 0.12032953]


In [61]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='pietrodimarco', api_key='sKrtjrFshiLPUvAGKx8Q')

# Create a trace
trace = go.Scatter(
    x = trainX,
    y = trainY,
    mode = 'markers',
    name='Collected value'
)
trace1 = go.Scatter(
    x = trainX,
    y = pred,
    name='Regression line'
)

data = [trace,trace1]

layout = go.Layout(
    xaxis=dict(
        title='ID',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        tickfont=dict(
            size=14,
            color='white'
        ),
        showticklabels=True,
        tickangle=45,
        showexponent='All'
    ),
    yaxis=dict(
        title='MovementTime',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        tickfont=dict(
            size=14,
            color='white'
        ),
        showticklabels=True,
        tickangle=45,
        showexponent='All'
    ),
    
    barmode='group',
    bargroupgap=0.01,
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~pietrodimarco/2.embed" height="525px" width="100%"></iframe>

In [67]:

l=list()
i=0
for row in mean.itertuples():
    l.append(go.Bar(x=row[1],y=row[2],text=row[2],textposition = 'auto',name=['test: '+str(i)],width = [0.1]))
    i+=1


data = l
layout = go.Layout(
    xaxis=dict(
        title='ID',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        tickfont=dict(
            size=14,
            color='white'
        ),
        showticklabels=True,
        tickangle=45,
        showexponent='All'
    ),
    yaxis=dict(
        title='Throughput',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        tickfont=dict(
            size=14,
            color='white'
        ),
        showticklabels=True,
        tickangle=45,
        showexponent='All'
    ),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~pietrodimarco/2.embed" height="525px" width="100%"></iframe>